<a href="https://colab.research.google.com/github/joyce-leesw/Webscraping/blob/main/top_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import lxml
import re
import numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/gdrive/')

# base_path = "gdrive/MyDrive"
# os.chdir(base_path)

from bs4 import BeautifulSoup
from requests import get

Mounted at /content/gdrive/


In [3]:
os.listdir()

['.config', 'gdrive', 'sample_data']

In [8]:
class IMDB(object):
	"""docstring for IMDB"""
	def __init__(self, url):
		super(IMDB, self).__init__()
		page = get(url)

		self.soup = BeautifulSoup(page.content, 'lxml')

	def articleTitle(self):
		return self.soup.find("h1", class_="header").text.replace("\n","")

	def bodyContent(self):
		content = self.soup.find(id="main")
		return content.find_all("div", class_="lister-item mode-advanced")

	def movieData(self):
		movieFrame = self.bodyContent()
		movieTitle = []
		movieDate = []
		movieRunTime = []
		movieGenre = []
		movieRating = []
		movieDirector = []
		movieStars = []
		for movie in movieFrame:
			movieFirstLine = movie.find("h3", class_="lister-item-header")
			movieTitle.append(movieFirstLine.find("a").text)
			movieDate.append(re.sub(r"[()]","", movieFirstLine.find_all("span")[-1].text))
			try:
				movieRunTime.append(movie.find("span", class_="runtime").text[:-4])
			except:
				movieRunTime.append(np.nan)
			try:
				movieGenre.append(movie.find("span", class_="genre").text.rstrip().replace("\n","").split(","))
			except:
					movieGenre.append(np.nan)
			try:
				movieRating.append(movie.find("strong").text)
			except:
				movieRating.append(np.nan)
			
			shit = movie.find_all("p", class_="")
			try: 
				# shit[1].text.replace("\n","").split('|')[1])
				DirandStar = shit[1].text.replace("\n","").split('|')
				# Stars_string = DirandStar[1].replace("Stars:","")
				Stars_string = DirandStar[1].strip().replace(":",",")
				movieCast_list = Stars_string.split(",")[1:]
				movieStars.append(movieCast_list)
			except:
				movieStars.append(np.nan)

			movieNumbers = movie.find_all("span", attrs={"name": "nv"})
	
		movieData = pd.DataFrame({ "Title": movieTitle, "Release_Year": movieDate, "Genre": movieGenre, "Runtime": movieRunTime, "Ratings": movieRating,"Stars": movieStars,})
		
		movieData = [movieTitle, movieDate, movieRunTime, movieGenre, movieRating, movieStars]
		return movieData

In [9]:
array_popularity_asc = []

In [10]:
# cant be greater than 40
MAX_PAGE = 40

imageArr = []

year_details = '&year=1931-01-01,2016-12-31'

for i in range(0,MAX_PAGE):
    totalRecords = 0 if i==0 else (250*i)+1
    url_pop_asc = f'https://www.imdb.com/search/title/?title_type=feature{year_details}&languages=en&count=250&start={totalRecords}&sort=moviemeter,asc'
    array_popularity_asc.append(url_pop_asc)


In [11]:
all_data_urls = array_popularity_asc
movieTitle_array, movieDate_array, movieRunTime_array, movieGenre_array, movieRating_array, movieStars_array = [],[],[],[],[],[]

for i in range(len(all_data_urls)):
    id1 = IMDB(all_data_urls[i])
    movieTitle_array = movieTitle_array + id1.movieData()[0]
    movieDate_array = movieDate_array + id1.movieData()[1]
    movieRunTime_array = movieRunTime_array + id1.movieData()[2]
    movieGenre_array = movieGenre_array + id1.movieData()[3]
    movieRating_array = movieRating_array + id1.movieData()[4]
    movieStars_array = movieStars_array + id1.movieData()[5]


messy_movieDf = pd.DataFrame({
    "Title": movieTitle_array,
    "Release_Year": movieDate_array,
    "Run_Time": movieRunTime_array,
    "Genre": movieGenre_array,
    "Rating": movieRating_array,
    "Stars": movieStars_array,
})

In [12]:
messy_movieDf.shape

(10000, 6)

In [13]:
# movieDf = messy_movieDf.loc[messy_movieDf.astype(str).drop_duplicates().index]
# movieDf

In [14]:
# # display(movieDf)
# string = ""
# for i in list(movieDf["Stars"].iloc[int(0.5*len(movieDf)):]):
#   string = string+f"{i}, "
# # type(movieDf)
# print(string)